# L6a: The Multiplicative Weights Algorithm (MWA)

---

This lecture will discuss the Multiplicative Weights Algorithm (MWA) and its applications in online learning. The key ideas discussed in this lecture are:
* [Online Learning](https://en.wikipedia.org/wiki/Online_machine_learning) is a type of machine learning where the model learns from a data stream. Thus, unlike traditional supervised machine learning, the model is not trained on a fixed dataset. Instead, it learns from new data as it arrives. This idea dates [back to the 1950s and has been studied extensively in the context of game theory, optimization, and machine learning](https://github.com/varnerlab/CHEME-5820-Lectures-Spring-2025/blob/main/lectures/week-6/L6a/docs/Orabona-OnlineLearning-arXiv-2019.pdf).
* [Regret minimization](https://github.com/varnerlab/CHEME-5820-Lectures-Spring-2025/blob/main/lectures/week-6/L6a/docs/Orabona-OnlineLearning-AnnRev-2021.pdf) in online learning involves designing algorithms that aim to minimize the difference between the algorithm's performance and that of the best possible strategy in hindsight, often measured as cumulative regret over time. This approach is particularly useful in sequential decision-making scenarios where the environment is dynamic or uncertain, such as financial portfolio optimization or strategic games.
* [The Multiplicative Weights Algorithm (MWA)](https://en.wikipedia.org/wiki/Multiplicative_weight_update_method) is a simple and powerful algorithm for online learning. The MWA is a type of sequential prediction algorithm that updates the weights of experts based on their performance on past predictions. The key idea is to assign higher weights to experts who perform well and lower weights to experts who perform poorly. This allows the algorithm to adapt to changing data distributions and learn from its mistakes.

Today, we'll borrow notes from several sources: [Arora et al., The Multiplicative Weights Update Method: A Meta-Algorithm and Applications, Theory of Computing, Volume 8 (2012), pp. 121–164](https://theoryofcomputing.org/articles/v008a006/v008a006.pdf) and the [15-859 CMU Lecture 16](https://www.cs.cmu.edu/afs/cs.cmu.edu/academic/class/15859-f11/www/notes/lecture16.pdf) and [15-850 CMU Lecture 17](https://www.cs.cmu.edu/afs/cs.cmu.edu/academic/class/15859-f11/www/notes/lecture17.pdf). In addition, we've linked to a few other sources linked in this week's module in canvas.

---

## Online Learning and Regret Minimization
Online learning algorithms typically consist of the following components:
* __Sequential data processing__: These algorithms process data sequentially, updating the model incrementally as new data becomes available.
* __Prediction mechanism__: The algorithm makes predictions for each incoming data point based on its current model, often using techniques like online convex optimization or recursive least squares.
* __Immediate feedback and model update__: After each prediction, the algorithm receives immediate feedback and updates its model accordingly, minimizing the discrepancy between predicted and actual outcomes

Suppose we are playing a number guess game where we have to predict a number over $T$ rounds. 

For $t = 1, 2, ..., T$:
1. An _adversary_ picks a number $y_t$ from the set $\{1, 2, ..., N\}$ and keeps it secret.
2. The _aggregator_ (us) make a prediction $p_t$ for the number.
3. The adversary reveals the true number $y_t$, and we receive feedback using a loss function $l_t(p_t, y_{t})$, which measures the discrepancy between our prediction and the true number.

The cumulative difference between what we selected and the best possible strategy in hindsight is called the _regret_:
$$
\begin{equation*}
\text{Regret} = \underbrace{\sum_{t=1}^{T} l_t(p_t, y_t)}_{\text{aggregator}} - \underbrace{\min_{u} \sum_{t=1}^{T} l_t(u,y_t)}_{\text{best possible strategy in hindsight}}
\end{equation*}
$$

Online learning aims to design algorithms that _minimize cumulative regret_, ensuring that our online learning algorithm performs _nearly_ as well as the best possible strategy in hindsight.

## Weighted Majority Algorithm
Let's start with the Weighted Majority Algorithm developed by Littlestone and Warmuth in 1994:
* [Littlestone, N., & Warmuth, M. K. (1994). The weighted majority algorithm. Information and Computation, 108(2), 212-261.](https://www.sciencedirect.com/science/article/pii/S0890540184710091)

We illustrate this approach with an example found in [Arora et al., 2005, Princeton](https://github.com/varnerlab/CHEME-5820-Lectures-Spring-2025/blob/main/lectures/week-6/L6a/docs/Arora-MWsurvey-CS-Princeton.pdf), which is a _prediction from expert advice_ problem. Let's take a look at the framing: 

* __Game__: Suppose we want to predict the daily movement of a stock price, i.e., whether it will go `{up | down}` on the day; in this case, we are predicting a _binary_ outcome which we encode as  `{1 | -1}`. We predict whether the price will go up or down each morning. At the market close, we find out the `true` movement of the stock price for the day. _If our prediction is incorrect, we lose a dollar_. Thus, our objective is to _minimize_ our losses throughout the game. However, we don't do this on our own. In making our predictions, we watch the predictions of $n$ experts (whose predictions may correlated and who may or may be correct). 
* __Goal__: The weighted majority algorithm (us) must limit our losses to approximately the same as the _best_ expert. However, we do this without knowing which expert is the best, i.e., it is not known until the end of the sequence of days who the best expert is, but we must make decisions each day. The algorithm does this by maintaining a weighting of the experts. Initially, all have equal weight. As time passes, some experts are seen as making better predictions than others, and the algorithm increases their weight proportionately.

We play this game between an omniscient _adversary_ (nature, i.e., the market) and an _aggregator_ (us) who $n$ experts advise; we select $n$ as odd to avoid ties. The game proceeds in $T$ rounds $t = 1, 2, \ldots, T$. During each round the aggregator (us) makes a _binary_ decision $y_t \in \{-1, 1\}$, and the adversary (market) reveals the true outcome $y_t$. Initially, the experts have weights $\left\{w_{i}^{(1)} = 1 \mid i = 1, 2, \ldots, n\right\}$. 

### Algorithm
For each round $t=1,2,\dots,T$:
1. The aggregator (us) makes a prediction $y_t \in \{-1, 1\}$ based on the weighted majority of the experts' predictions. If the total weight of all experts predicting `1` at time $t$ is $w^{(t)}\geq\sum_{i}w_{i}^{(t)}/2$, then the aggregator predicts `1`, otherwise it predicts `-1`.
2. The adversary (market) reveals the actual outcome $y_t \in \{-1, 1\}$.
3. We decrease the weights of the experts who predicted incorrectly. For each expert $i$ who predicted incorrectly, we update the weight: $w_{i}^{(t+1)} = w_{i}^{(t)}(1-\epsilon)$, where $0<\epsilon\leq{1/2}$ is a learning rate parameter.

__Theorem__: The weighted majority algorithm has the following theoretical guarantee (which bounds the number of mistakes the aggregator makes). Let $m_{i}^{(t)}$ be the number of mistakes made by expert $i$ up to time $t$ and $m^{(t)}$ be the total number of mistakes made by the aggregator (us). Then, for every expert $i$ and the aggregator, we have:
$$
\begin{align*}
m^{(t)} \leq \frac{2\ln(n)}{\epsilon} + 2\left(1+\epsilon\right)m_{i}^{(t)}
\end{align*}
$$

The [proof of this theorem can be found here](https://github.com/varnerlab/CHEME-5820-Lectures-Spring-2025/blob/main/lectures/week-6/L6a/docs/Arora-MWsurvey-CS-Princeton.pdf). The MW algorithm is deterministic and has a simple implementation (any randomness comes from the adversary). 

## Hedge strategy
Next, let's look at a modification to the weighted majority algorithm called the _Hedge strategy_ introduced initially by Freund and Schapire in 1997:
* [Yoav Freund and Robert E. Schapire. A decision-theoretic generalization of online learning and an application to boosting. J. Comput. Syst. Sci., 55:119–139, August 1997](https://www.cis.upenn.edu/~mkearns/teaching/COLT/adaboost.pdf)

* __Game__: In the _Hedge strategy_, the aggregator (us) has $N$ experts (or options) from which to choose. At each time step $t$, the aggregator (us) decides on a resource distribution
$\mathbf{p}^{(t)} = \left\{p_{1}^{(t)}, p_{2}^{(t)}, \ldots, p_{N}^{(t)}\right\}$ over the experts; where $p_{i}^{(t)}\geq{0}$ is the fraction allocated to expert (option) $i$ at time $t$ assuming full allocation $\sum_{i=1}^{N}p_{i}^{(t)} = 1$. Each strategy suffers some loss $l_{i}^{(t)}$ at time $t$, which is determined by comparing the prediction of expert $i$ to the true outcome provided by the adversary (market). The total loss (also called the _mixture loss_) suffered by the aggregator (us) at time $t$ is given by:
$$
\begin{align*}
L^{(t)} = \sum_{i=1}^{N}p_{i}^{(t)}l_{i}^{(t)}
\end{align*}
$$
The goal of the aggregator (us) is to minimize its cumulative loss $L_{A}$ throughout the game, relative to the best expert:
$$
\begin{align*}
L_{A} = \sum_{t=1}^{T}L^{(t)} - \min_{i}\left(\sum_{t=1}^{T}l_{i}^{(t)}\right)
\end{align*}
$$

### Algorithm
Let's look at the algorithm. For $t=1,2,\dots,T$:
1. The aggregator (us) computes a distribution $\mathbf{p}^{(t)} = \left\{p_{1}^{(t)}, p_{2}^{(t)}, \ldots, p_{N}^{(t)}\right\}$ over the experts, where $p_{i}^{(t)}\geq{0}$ is the fraction of the resources allocated to expert $i$ at time $t$ and we have full allocation $\sum_{i=1}^{N}p_{i}^{(t)} = 1$. The distribution is computed as:
$$
\begin{align*}
p_{i}^{(t)} = w_{i}^{(t)}/\sum_{j=1}^{N}w_{j}^{(t)}
\end{align*}
$$
2. The experts $i$ make their predictions $y_{i}^{(t)}\in\{-1, 1\}$ (or some other outcome), and the adversary (market) reveals the actual outcome $y_{t}\in\{-1, 1\}$.
3. We compute the loss for each expert $i$ as:
$$
\begin{align*}
l_{i}^{(t)} = \left\{
\begin{array}{ll}
1 & \text{if } y_{t}\neq{y_{i}^{(t)}}\\
0 & \text{otherwise}
\end{array}
\right.
\end{align*}
$$
4. The aggregator (us) updates the total loss $L^{(t)} = \sum_{i=1}^{N}p_{i}^{(t)}l_{i}^{(t)}$ and updates the weights of the experts as:
$$
\begin{align*}
w_{i}^{(t+1)} = w_{i}^{(t)}\cdot\beta^{l_{i}^{(t)}}
\end{align*}
$$
where $\beta>0$ is a learning hyperparameter. [There are several interesting theoretical results associated with the Hedge strategy, including some thoughts on the optimal choice of $\beta$ in the paper.](https://www.cis.upenn.edu/~mkearns/teaching/COLT/adaboost.pdf).

## Multiplicative Weights Algorithm (MWA)
The Multiplicative Weights Algorithm (MWA) is a generalization of the Weighted Majority Algorithm and the Hedge strategy. The MWA is a simple and robust online learning algorithm that can solve many optimization problems. 

* __Game__: Let $t = 1, 2, \ldots, T$ denote the current round of the game, and $i$ denote an expert advising us. In each round, we compute a _belief distribution_ $\mathbf{p}^{(t)} = \left\{p_{1}^{(t)}, p_{2}^{(t)}, \ldots, p_{N}^{(t)}\right\}$ over the experts, select a _random_ expert by sampling this distribution and use the selected expert to make a decision. At this point, the _adversary_ (nature) reveals the outcome, and we compute the cost of the decision we've made, where $\mathbf{m}^{(t)} = \left\{m_{1}^{(t)}, m_{2}^{(t)}, \ldots, m_{N}^{(t)}\right\}$ is the overall cost vector and $m_{i}^{(t)}$ is the cost of expert decision $i$ at time $t$. Here, we assume that the costs are in the range $m_{i}^{(t)}\in[-1, 1]$. Then, the total expected loss at time $t$ is: $L^{(t)} = \sum_{i=1}^{N}p_{i}^{(t)}m_{i}^{(t)}$, while the overall loss experienced by the _aggregator_ (at the end of the game) is; $L_{A} = \sum_{t=1}^{T}L^{(t)}$.
* __Goal__: The goal of the aggregator (us) is to minimize the total expected loss $L_{A}$ throughout the game, such that we do not experience a loss that is significantly worse than the best decision in hindsight, i.e., $\min_{i}\left(\sum_{t=1}^{T}m_{i}^{(t)}\right)$.

#### Algorithm
Fix a learning rate $\eta\leq{1}/{2}$, for each expert initialize the weight $w_{i}^{(1)} = 1$.

For $t=1,2,\dots,T$:
1. Chose expert $i$ with probability $p_{i}^{(t)} = w_{i}^{(t)}/\sum_{j=1}^{N}w_{j}^{(t)}$. Ask expert $i$ what the outcome of the experiment should be, denote this as: $y_{i}^{(t)}$.
2. The adversary (nature) reveals the true outcome $y_{t}$. Compute the cost of the following expert $i$. If the expert predicted the outcome of the experiment correctly, then the cost is $m_{i}^{(t)}$ = `-1`, otherwise the cost for an incorrect prediction is $m_{i}^{(t)}$ = `1`. Note: the costs can be in the range $m_{i}^{(t)}\in[-1, 1]$.
3. Update the weights of expert $i$ as:
$$
\begin{align*}
w_{i}^{(t+1)} = w_{i}^{(t)}\cdot\left(1-\eta\cdot{m_{i}^{(t)}}\right)
\end{align*}
$$

__Theorem__: The MWA has the following theoretical guarantee. Assume all costs are in the range $m_{i}^{(t)}\in[-1, 1]$ and $\eta\leq{1}/{2}$. Then the Multiplicative Weights Algorithm (MWA) guarantees that after $T$ rounds, for any decision $i$, we have:
$$
\begin{align*}
\sum_{t=1}^{T}\mathbf{m}^{(t)}\cdot\mathbf{p}^{(t)} & \leq \sum_{t = 1}^{T}m_{i}^{(t)}+\eta\sum_{t=1}^{T}|m_{i}^{(t)}|+\frac{\ln{n}}{\eta}
\end{align*}
$$

# Today?
That's a wrap! What are some of the interesting things we discussed today?